## 파인튜닝 마친 모델로 문장 생성

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)  

Mounted at /gdrive


In [3]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 53.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 70.2 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=b92a33d3224bc34867a6fdffb71ab6d142fe69b38b88ea59ccf1a956c6cc2109
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c1

### 인퍼런스 설정

In [4]:
from ratsnlp.nlpbook.generation import GenerationDeployArguments
args = GenerationDeployArguments(
    pretrained_model_name="skt/kogpt2-base-v2",  #파인튜닝 때 사용한 프리트레인 마친 언어 모델 이름
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-generation",   #파인튜닝한 모델의 저장 위치
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-generation/epoch=1-val_loss=2.29.ckpt


### 모델 로딩

파인튜닝을 마친 GPT2 모델과 토크나이저를 읽어들임

- 모델 불러오기

In [6]:
import torch
from transformers import GPT2Config, GPT2LMHeadModel
pretrained_model_config = GPT2Config.from_pretrained(
    args.pretrained_model_name,
)

model = GPT2LMHeadModel(pretrained_model_config)
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu"),
)

model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


- 토크나이저 초기화

In [7]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>",
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


### 인퍼런스 함수 선언

In [14]:
def inference_fn(
        prompt,
        min_length=10,
        max_length=50,
        top_p=1.0,
        top_k=50,
        repetition_penalty=1.0,
        no_repeat_ngram_size=0,
        temperature=1.0,
):
    try:
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        with torch.no_grad():
            generated_ids = model.generate(
                input_ids,
                do_sample=True,
                top_p=float(top_p),
                top_k=int(top_k),
                min_length=int(min_length),
                max_length=int(max_length),
                repetition_penalty=float(repetition_penalty),
                no_repeat_ngram_size=int(no_repeat_ngram_size),
                temperature=float(temperature),
           )
        generated_sentence = tokenizer.decode([el.item() for el in generated_ids[0]])
    except:
        generated_sentence = """처리 중 오류가 발생했습니다. <br>
            변수의 입력 범위를 확인하세요. <br><br> 
            min_length: 1 이상의 정수 <br>
            max_length: 1 이상의 정수 <br>
            top-p: 0 이상 1 이하의 실수 <br>
            top-k: 1 이상의 정수 <br>
            repetition_penalty: 1 이상의 실수 <br>
            no_repeat_ngram_size: 1 이상의 정수 <br>
            temperature: 0 이상의 실수
            """
    return {
        'result': generated_sentence,
    }

In [13]:
inference_fn(
        prompt='영화를 보는 것은',
        min_length=10,
        max_length=50,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.1,
        no_repeat_ngram_size=3,
        temperature=0.5,
)

{'result': '영화를 보는 것은 시간 낭비인 영화. 이 영화는 그냥 킬링타임용으로 봐야 한다.</s>'}

### 웹서비스

In [10]:
!mkdir /root/.ngrok2 && echo "authtoken: 2K7HpOJzbZrOEtWlKir6NjCmzOj_7oQuKdBKnQ1fkwDuXUW8o" > /root/.ngrok2/ngrok.yml

In [11]:
!ls /root -al

total 64
drwx------ 1 root root 4096 Jan 18 03:02 .
drwxr-xr-x 1 root root 4096 Jan 18 02:30 ..
-r-xr-xr-x 1 root root 1169 Jan  1  2000 .bashrc
drwxr-xr-x 1 root root 4096 Jan 18 02:38 .cache
drwx------ 1 root root 4096 Jan 18 02:30 .config
drwxr-xr-x 5 root root 4096 Jan 13 14:45 .ipython
drwx------ 2 root root 4096 Jan 13 14:45 .jupyter
drwxr-xr-x 2 root root 4096 Jan 13 14:44 .keras
drwxr-xr-x 1 root root 4096 Jan 13 14:45 .local
drwxr-xr-x 2 root root 4096 Jan 18 03:02 .ngrok2
drwxr-xr-x 4 root root 4096 Jan 13 14:46 .npm
-rw-r--r-- 1 root root  161 Dec  5  2019 .profile
-r-xr-xr-x 1 root root  254 Jan  1  2000 .tmux.conf
-rw-r--r-- 1 root root  165 Jan 13 14:45 .wget-hsts


In [12]:
from ratsnlp.nlpbook.generation import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.generation.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://cfbf-35-185-82-179.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [18/Jan/2023 03:03:08] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2023 03:03:08] "GET /favicon.ico HTTP/1.1" 404 -
